# Modelo de Regresión Lineal

## Introducción

##### En el siguiente proyecto estaré desarrollando un modelo de machine learning para una compañia de petróleo que desea saber cual es el mejor lugar para abrir 200 nuevos pozos. Estaré estudiando los beneficios y riesgos potenciales basados en los datos sobre muestras de crudo en tres regiones distintas y el modelo deberá seleccionar la región con el mayor márgen de beneficio para la empresa.

### Tabla de contenido:

## 1. Inicialización

In [2]:
#importar librerías
import pandas as pd
from sklearn.metrics import recall_score, precision_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib as plt 
from sklearn.linear_model import LinearRegression
